In [ ]:
import sys
sys.path.append("/Users/andrei/Desktop/Repos/BBAttacks/attacks/")
sys.path.append("/Users/andrei/Desktop/Repos/BBAttacks/utils/")

import utils
from data_manager import load_data

# Black Box Attacks
import random_noise
import EvoStrategy
import SimbaWrapper

In [ ]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
from tensorflow.keras.datasets import cifar100
from tensorflow.keras import backend as K
from tensorflow import keras as keras
import tensorflow as tf
import json
import time
import datetime
from pathlib import Path
from tqdm import tqdm
import sys
import lightgbm as lgb
import optuna
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
cifar_100_classes = ["apple", "aquarium_fish", "baby", "bear", "beaver", "bed", "bee", "beetle", "bicycle", "bottle", "bowl", "boy", "bridge", "bus", "butterfly", "camel", "can", "castle", "caterpillar", "cattle", "chair", "chimpanzee", "clock", "cloud", "cockroach", "couch", "cra", "crocodile", "cup", "dinosaur", "dolphin", "elephant", "flatfish", "forest", "fox", "girl", "hamster", "house", "kangaroo", "keyboard", "lamp", "lawn_mower", "leopard", "lion", "lizard", "lobster", "man", "maple_tree", "motorcycle", "mountain", "mouse", "mushroom", "oak_tree", "orange", "orchid", "otter", "palm_tree", "pear", "pickup_truck", "pine_tree", "plain", "plate", "poppy", "porcupine", "possum", "rabbit", "raccoon", "ray", "road", "rocket", "rose", "sea", "seal", "shark", "shrew", "skunk", "skyscraper", "snail", "snake", "spider", "squirrel", "streetcar", "sunflower", "sweet_pepper", "table", "tank", "telephone", "television", "tiger", "tractor", "train", "trout", "tulip", "turtle", "wardrobe", "whale", "willow_tree", "wolf", "woman", "worm"]

<h3> Loading model and data </h3>

In [ ]:
CIFAR100_VGG_PATH = "/Users/andrei/Desktop/Repos/cifar-vgg/"
sys.path.append(CIFAR100_VGG_PATH)
import cifar100vgg

In [ ]:
importlib.reload(cifar100vgg)
model = cifar100vgg.cifar100vgg(train=False)

NUM_CLASSES = 100

In [ ]:
LOAD_DATA = True
SAVE_DATA = False

In [ ]:
if LOAD_DATA:
    (x_train, y_train), (x_test, y_test) = cifar100.load_data()

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
    y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)
    
    if SAVE_DATA:
        np.save("x_test.npy", x_test)
        np.save("y_test.npy", y_test)

In [ ]:
x_train = x_train.astype('int')
x_test = x_test.astype('int')

In [ ]:
import gc
gc.collect()

In [ ]:
%%time
preds_train = model.predict(x_train)

In [ ]:
preds_train_labels = np.argmax(preds_train, axis=1)

In [ ]:
true_train_labels = np.argmax(y_train, axis=1)

In [ ]:
acc_train = np.mean(true_train_labels == preds_train_labels)

In [ ]:
acc_train

In [ ]:
preds = model.predict(x_test)

In [ ]:
pred_labels = np.argmax(preds, axis=1)
true_labels = np.argmax(y_test, axis=1)
acc = np.sum(pred_labels == true_labels) / len(y_test)
print("Accuracy: ", acc)

In [ ]:
# Extracting correctly classified test samples
x_test_correct = x_test[pred_labels == true_labels]
y_test_correct = y_test[pred_labels == true_labels]

In [ ]:
test_class_correct_mask = (pred_labels == true_labels)

<h3> Intermediary layers utils </h3>

In [ ]:
def get_intermediate_outputs(model, batch_input):
    intermediate_layer_names = [layer.name for layer in model.model.layers] 
    
    intermediate_layers_model = [
        keras.Model(
            inputs=model.model.input, 
            outputs=model.model.get_layer(layer_name).output,
        ) for layer_name in intermediate_layer_names
    ]
    
    normalized_batch = model.normalize_production(batch_input)
        
    intermediate_batch_outputs = [intermediate_layer_model(normalized_batch) for intermediate_layer_model in intermediate_layers_model]
    
    # return np.array(intermediate_batch_outputs)
    return intermediate_batch_outputs

In [ ]:
def plot_layer_to_max_rel_change(interm_outputs_1, interm_outputs_2, show=True):
    interm_outputs_delta = []
    
    for layer in range(len(interm_outputs_1)):
        interm_outputs_delta.append(interm_outputs_2[layer] - interm_outputs_1[layer])
    
    layer_to_max_rel_change = []
    for layer in range(len(interm_outputs_1)):
        max_change_curr_layer = (np.sum(np.abs(interm_outputs_delta[layer][0]))/np.sum(np.abs(interm_outputs_1[layer][0])))
        layer_to_max_rel_change.append(max_change_curr_layer)
    
    if show:
        plt.figure(figsize=(8,6))
        plt.plot(layer_to_max_rel_change)
        plt.show()
    
    return layer_to_max_rel_change

In [ ]:
def get_layer_outputs(model, batch_input, layer_index):
    intermediate_layer_names = [layer.name for layer in model.model.layers] 
    
    layer_name = intermediate_layer_names[layer_index]
    
    intermediate_layer_model = keras.Model(
        inputs=model.model.input, 
        outputs=model.model.get_layer(layer_name).output,
    )
    
    normalized_batch = model.normalize_production(batch_input)
    
    intermediate_batch_outputs = intermediate_layer_model(normalized_batch)
    
    # return np.array(intermediate_batch_outputs)
    return intermediate_batch_outputs

<h3> Run EvoBA against the neural network </h3>

In [ ]:
importlib.reload(EvoStrategy)

In [ ]:
GENERATION_SIZE = 30
PIXEL_COUNT = 1
STEPS = 80

adv_evo_strategy = {}

# SIZE = len(x_test_correct)
SIZE = 100
VERBOSE = False

perturbed_images = 0
failed_images = 0
queries = []


for index in tqdm(range(SIZE)):
    if test_class_correct_mask[index]:
        img = x_test[index]
        label = np.argmax(y_test[index])
        adv_evo_strategy[index] = EvoStrategy.AdversarialPerturbationEvoStraegy(
            model=model,
            img=img,
            label=label,
            generation_size=GENERATION_SIZE, 
            one_step_perturbation_pixel_count=PIXEL_COUNT,
            verbose=VERBOSE,
            zero_one_scale=False
        )
        no_steps = adv_evo_strategy[index].run_adversarial_attack(steps=STEPS)
        if adv_evo_strategy[index].is_perturbed() and no_steps > 0:
            perturbed_images += 1
            queries.append(adv_evo_strategy[index].queries)
        else:
            failed_images += 1
            queries.append(-1)

<h3> Plot relative perturbation per layer of the EvoBA adversarial samples </h3>

In [ ]:
interm_outputs_modif = {}
interm_outputs = {}

In [ ]:
plot_arrays = {}

In [ ]:
SAMPLE_SIZE_INTERM = SIZE

In [ ]:
# for index_diff in range(RESTART_INDEX, SIZE):
for index_diff in tqdm(range(SAMPLE_SIZE_INTERM)):
    if test_class_correct_mask[index_diff]:
        modif_img = adv_evo_strategy[index_diff].get_best_candidate()
        orig_img = x_test[index_diff]

        # modif_img_normalized_array = model.normalize_production(np.array([modif_img]).astype(float))
        # orig_img_normalized_array = model.normalize_production(np.array([orig_img]).astype(float))

        interm_outputs_modif[index_diff] = get_intermediate_outputs(model, np.array([modif_img]).astype(float))
        interm_outputs[index_diff] = get_intermediate_outputs(model, np.array([orig_img]).astype(float))

        plot_arrays[index_diff] = plot_layer_to_max_rel_change(
            interm_outputs[index_diff], 
            interm_outputs_modif[index_diff],
            show=False
        )

In [ ]:
# for index_diff in tqdm(range(RESTART_INDEX, SIZE)):
for index_diff in range(SAMPLE_SIZE_INTERM):
    if test_class_correct_mask[index_diff]:
        plot_arrays[index_diff] = plot_layer_to_max_rel_change(
            interm_outputs[index_diff], 
            interm_outputs_modif[index_diff],
            show=False
        )

In [ ]:
COLS = 2
ROWS = 3

FONTSIZE = 32
FONTSIZE_AXIS = 28
FONTSIZE_LEGEND = 24

fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * 12,COLS * 10), )
fig.tight_layout(pad=15.0)

for i in range(ROWS):
    for j in range(COLS):
        
        avg_changes_linear = []
        indices_linear = []

        avg_changes_activation = []
        indices_activation = []
        
        img_index = list(plot_arrays.keys())[i * COLS + j]
        
        for index, layer in enumerate(model.model.layers[:-1]):
            if layer.name.startswith("conv") or layer.name.startswith("dense"):
                avg_changes_linear.append(plot_arrays[img_index][index])
                indices_linear.append(index)
            elif layer.name.startswith("activ"):
                avg_changes_activation.append(plot_arrays[img_index][index])        
                indices_activation.append(index)
                
        axs[i,j].set_title(f"Image {img_index}", fontsize=FONTSIZE)
        axs[i,j].set_ylim(0, 1.2)
        axs[i,j].plot(plot_arrays[img_index][:-1])
        axs[i,j].set_xlabel("Layer index", fontsize=FONTSIZE)
        axs[i,j].set_ylabel("Relative change", fontsize=FONTSIZE)
        
        axs[i,j].xaxis.set_tick_params(labelsize=FONTSIZE_AXIS)
        axs[i,j].yaxis.set_tick_params(labelsize=FONTSIZE_AXIS)

        axs[i,j].scatter(indices_linear, avg_changes_linear, c="orange", label="Conv or dense layers")
        axs[i,j].scatter(indices_activation, avg_changes_activation, c="red", label="Activation layers")
        axs[i,j].legend(fontsize=FONTSIZE_LEGEND)

plt.xticks(fontsize=FONTSIZE_AXIS)
plt.yticks(fontsize=FONTSIZE_AXIS)
plt.show()

In [ ]:
avg_changes = np.mean(np.array(list(plot_arrays.values())), axis=0)[:-1]

In [ ]:
top_changes_in_layers_list = [model.model.layers[x].get_config() for x in list(reversed(np.argsort(avg_changes)))]

In [ ]:
avg_changes_linear = []
indices_linear = []

avg_changes_activation = []
indices_activation = []

for index, layer in enumerate(model.model.layers[:-1]):
    if layer.name.startswith("conv") or layer.name.startswith("dense"):
        avg_changes_linear.append(avg_changes[index])
        indices_linear.append(index)
    elif layer.name.startswith("activ"):
        avg_changes_activation.append(avg_changes[index])        
        indices_activation.append(index)

In [ ]:
FONTSIZE = 28
FONTSIZE_AXIS = 24
FONTSIZE_LEGEND = 24
plt.figure(figsize=(20,10))
plt.xlabel("Layer index", fontsize=FONTSIZE)
plt.ylabel("Relative average change", fontsize=FONTSIZE)
plt.xticks(fontsize=FONTSIZE_AXIS)
plt.yticks(fontsize=FONTSIZE_AXIS)
plt.plot(avg_changes)
plt.scatter(indices_linear, avg_changes_linear, c="orange", label="Conv or dense layers")
plt.scatter(indices_activation, avg_changes_activation, c="red", label="Activation layers")
plt.legend(fontsize=FONTSIZE_LEGEND)

<h3> Train LGB on intermediary output </h3>

In [ ]:
layer_names = [model.model.layers[x].get_config()['name'] for x in range(len(model.model.layers))]

In [ ]:
LAYER_IDX = 40

SLICE_SIZE = 200

train_features = []
for i in tqdm(range(len(x_train) // SLICE_SIZE)):
    left = i * SLICE_SIZE
    right = left + SLICE_SIZE
    batch_layer_outputs = get_layer_outputs(model, x_train[left:right], LAYER_IDX)
    batch_layer_outputs = np.reshape(batch_layer_outputs, (SLICE_SIZE, -1))
    train_features.append(batch_layer_outputs)

train_features = np.vstack(train_features)

In [ ]:
X_train_stack, X_val, y_train_stack, y_val = train_test_split(train_features, y_train, test_size=0.1, random_state=42)

In [ ]:
y_train_stack = np.argmax(y_train_stack, axis=1)
y_val = np.argmax(y_val, axis=1)

In [ ]:
RUN_OPTUNA = False

In [ ]:
best_params = {
    'lambda_l1': 0.8021389627616706, 
    'lambda_l2': 9.818860936043825, 
    'num_leaves': 135, 
    'feature_fraction': 0.6016921652574784, 
    'bagging_fraction': 0.9902711576251494, 
    'bagging_freq': 1, 
    'min_child_samples': 81
}

In [ ]:
def objective(trial):
    train_x, test_x, train_y, test_y = X_train_stack, X_val, y_train_stack, y_val
    dtrain = lgb.Dataset(train_x, label=train_y)
 
    param = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'num_class': 100,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'verbose': -1,
    }
    gbm = lgb.train(param, dtrain, verbose_eval=False)
    preds = np.argmax(gbm.predict(test_x), axis=1)
    accuracy = sklearn.metrics.accuracy_score(test_y, preds)
    return accuracy
 
if RUN_OPTUNA:
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100, timeout=7200)

    print('Number of finished trials:', len(study.trials))
    print('Best trial:', study.best_trial.params)
    best_params = study.best_trial.params

In [ ]:
params = {
    **best_params,
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'num_class': 100,
    'verbose': -1,
}

In [ ]:
TUNE_NUM_ITERS = False
num_iters_full_train = 1000

In [ ]:
if TUNE_NUM_ITERS:
    dtrain = lgb.Dataset(X_train_stack, y_train_stack)
    dval = lgb.Dataset(X_val, y_val)

    lgb_stack_interm = lgb.train(params, train_set=dtrain, valid_sets=[dval], early_stopping_rounds=30, num_boost_round=1000, verbose_eval=10)
    
    num_iters_full_train = lgb_stack_interm.best_iteration

In [ ]:
X_train_stack_full = np.vstack([X_train_stack, X_val])
y_train_stack_full = np.concatenate((y_train_stack, y_val))

In [ ]:
dtrain_full = lgb.Dataset(X_train_stack_full, y_train_stack_full)
lgb_stack = lgb.train(params, train_set=dtrain_full)

In [ ]:
(np.argmax(lgb_stack.predict(X_val), axis=1) == y_val).mean()

<h3> Run stacked model on test data </h3>

In [ ]:
len(x_test)

In [ ]:
LAYER_IDX = 40

SLICE_SIZE = 200

test_features = []
for i in tqdm(range(len(x_test) // SLICE_SIZE)):
    left = i * SLICE_SIZE
    right = left + SLICE_SIZE
    batch_layer_outputs = get_layer_outputs(model, x_test[left:right], LAYER_IDX)
    batch_layer_outputs = np.reshape(batch_layer_outputs, (SLICE_SIZE, -1))
    test_features.append(batch_layer_outputs)

test_features = np.vstack(test_features)

In [ ]:
test_pred_lgb = lgb_stack.predict(test_features)

In [ ]:
stack_model_correct_test_indices = (np.argmax(test_pred_lgb, axis=1) == np.argmax(y_test, axis=1))

In [ ]:
stack_model_test_accuracy = stack_model_correct_test_indices.mean()

In [ ]:
stack_model_test_total_correct = stack_model_correct_test_indices.sum()

In [ ]:
x_test_correct_stack = x_test[stack_model_correct_test_indices]
y_test_correct_stack = y_test[stack_model_correct_test_indices]

In [ ]:
class StackedModel():
    def __init__(self, deep_model, layer_idx, lgb_model):
        self.deep_model = deep_model
        self.layer_idx = layer_idx
        self.lgb_model = lgb_model
        
        
    def predict_intermediary(self, batch_img):
        batch_size = len(batch_img)
        batch_layer_outputs = get_layer_outputs(self.deep_model, batch_img, self.layer_idx)
        batch_layer_outputs = np.reshape(batch_layer_outputs, (batch_size, -1))
        return batch_layer_outputs
    
    def predict(self, batch_img, verbose=False):
        batch_layer_outputs = self.predict_intermediary(batch_img)
        
        lgb_output = self.lgb_model.predict(batch_layer_outputs)
        return lgb_output

In [ ]:
stacked_model = StackedModel(model, LAYER_IDX, lgb_stack)

In [ ]:
len(model.model.layers)

<h3> Run EvoBA against the stacked model </h3>

In [ ]:
adv_evo_strategy_lgb = {}
queries_lgb = []
perturbed_images_lgb = 0
failed_images_lgb = 0

for index in tqdm(range(SIZE)):
    if stack_model_correct_test_indices[index]:
        img = x_test[index]
        label = np.argmax(y_test[index])
        adv_evo_strategy_lgb[index] = EvoStrategy.AdversarialPerturbationEvoStraegy(
            model=stacked_model,
            img=img,
            label=label,
            generation_size=GENERATION_SIZE, 
            one_step_perturbation_pixel_count=PIXEL_COUNT,
            verbose=VERBOSE,
            zero_one_scale=False
        )

        no_steps = adv_evo_strategy_lgb[index].run_adversarial_attack(steps=STEPS)
        if adv_evo_strategy_lgb[index].is_perturbed() and no_steps > 0:
            perturbed_images_lgb += 1
            queries_lgb.append(adv_evo_strategy_lgb[index].queries)
        else:
            failed_images_lgb += 1
            queries_lgb.append(-1)

        adv_evo_strategy_lgb[index].run_adversarial_attack(steps=STEPS)

In [ ]:
np.max(queries)

In [ ]:
# preds_stacked_model_on_test_correct = np.argmax(stacked_model.predict(x_test_correct_stack), axis=1)

In [ ]:
# lgb_acc_on_test_correct = sum(preds_stacked_model_on_test_correct == np.argmax(y_test_correct_stack, axis=1))

In [ ]:
# lgb_acc_on_test_correct

In [ ]:
# print(lgb_acc_on_test_correct)

In [ ]:
# extrapolate_total_perturbed_lgb = perturbed_images_lgb / lgb_acc_on_test_correct * stack_model_test_total_correct

In [ ]:
np.max(queries_lgb)

In [ ]:
avg_queries_succ = np.array(queries)[np.array(queries)>0].mean()
avg_queries_succ_lgb = np.array(queries_lgb)[np.array(queries_lgb)>0].mean()

print(
    "Stack model perturbed images:", perturbed_images_lgb, f"out of {perturbed_images_lgb + failed_images_lgb}",
    "correctly classified ones",
    f"i.e {perturbed_images_lgb / (perturbed_images_lgb + failed_images_lgb) * 100}%"
)
print(
    "Deep model perturbed images:", perturbed_images, f"out of {perturbed_images + failed_images}",
    f"i.e {perturbed_images / (perturbed_images + failed_images) * 100}%\n"
)

print("Stack model avg succ query:", avg_queries_succ)
print("Deep model avg succ query:", avg_queries_succ_lgb, "\n")

print("Stack model accuracy:", stack_model_test_accuracy)
print("Deep model accuracy:", acc, "\n")

<h4> Plot perturbed accuracy as a function of queries </h4>

In [ ]:
queries_lgb.sort()
queries.sort()

queries_lgb_array = np.array(queries_lgb)
queries_lgb_array = queries_lgb_array[queries_lgb_array > 0]

queries_array = np.array(queries)
queries_array = queries_array[queries_array > 0]

intermediary_accuracies = []
intermediary_lgb_accuracies = []
query_thresholds = []

for i in range(0, 2000, 50):
    curr_perturbed_count = np.sum(queries_array < i)
    curr_lgb_perturbed_count = np.sum(queries_lgb_array < i)
    
    intermediary_accuracies.append((perturbed_images + failed_images - curr_perturbed_count)/SIZE)
    intermediary_lgb_accuracies.append((perturbed_images_lgb + failed_images_lgb - curr_lgb_perturbed_count)/SIZE)
    query_thresholds.append(i)

In [ ]:
plt.plot(query_thresholds, intermediary_accuracies)
plt.plot(query_thresholds, intermediary_lgb_accuracies)

In [ ]:
len(adv_evo_strategy_lgb)

In [ ]:
common_perturbed_indices = []
for i in range(SIZE):
    if (i in adv_evo_strategy) and (i in adv_evo_strategy_lgb):
        if adv_evo_strategy[i].is_perturbed() and adv_evo_strategy_lgb[i].is_perturbed():
            common_perturbed_indices.append(i)

In [ ]:
adv_evo_strategy_deep_shared = [adv_evo_strategy[x] for x in common_perturbed_indices]
adv_evo_strategy_lgb_shared = [adv_evo_strategy_lgb[x] for x in common_perturbed_indices]

In [ ]:
evoba_stats_deep_shared = utils.get_evoba_stats(adv_evo_strategy_deep_shared)

In [ ]:
evoba_stats_lgb_shared = utils.get_evoba_stats(adv_evo_strategy_lgb_shared)

In [ ]:
utils.print_evoba_stats(evoba_stats_deep_shared)

In [ ]:
utils.print_evoba_stats(evoba_stats_lgb_shared)

In [ ]:
NUM_IMG_PLOT_COMPARISON = 12
fig, axs = plt.subplots(nrows=3, ncols=NUM_IMG_PLOT_COMPARISON, figsize=(3 * NUM_IMG_PLOT_COMPARISON, 11))
for i in range(NUM_IMG_PLOT_COMPARISON):
    IDX = random.randint(0, len(adv_evo_strategy_deep_shared))
    
    original_label = adv_evo_strategy_deep_shared[IDX].label
    
    axs[0][i].set_title(f"{cifar_100_classes[original_label]}")
    axs[0][i].imshow(adv_evo_strategy_deep_shared[IDX].img)
    
    axs[0][0].set_ylabel("Original image", fontsize=14)

    
    pred_class_lgb = stacked_model.predict(np.array([adv_evo_strategy_lgb_shared[IDX].get_best_candidate()]))[0]
    pred_class_lgb = np.argmax(pred_class_lgb)
    axs[1][i].set_title(f"{cifar_100_classes[pred_class_lgb]}")
    axs[1][i].imshow(adv_evo_strategy_lgb_shared[IDX].get_best_candidate())
    
    axs[1][0].set_ylabel("Ensemble perturbed iamge", fontsize=14)

    
    pred_class_deep = model.predict(np.array([adv_evo_strategy_deep_shared[IDX].get_best_candidate()]))[0]
    pred_class_deep = np.argmax(pred_class_deep)
    axs[2][i].set_title(f"{cifar_100_classes[pred_class_deep]}")
    axs[2][i].imshow(adv_evo_strategy_deep_shared[IDX].get_best_candidate())

    axs[2][0].set_ylabel("Deep perturbed iamge", fontsize=14)


<h4> Visualise robust vs non-robust image embeddings [deprecated for now, to review after changes in the way EvoBA loops are ran] </h4>

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
test_interm_features = stacked_model.predict_intermediary(x_test_correct)

In [ ]:
# TODO: try UMAP
X_embedded = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3).fit_transform(test_interm_features)

In [ ]:
X_embedded_x = [x[0] for x in X_embedded]
X_embedded_y = [x[1] for x in X_embedded]

In [ ]:
y_test_class = np.argmax(y_test_correct, axis=1)

In [ ]:
# Only plot first couple classes embeddings
class_plot_list = list(range(100))
class_to_idx_dict = {c: i for i, c in enumerate(class_plot_list)}
sample_mask = [x in class_plot_list for x in y_test_class]

X_embedded_x_sample = np.array(X_embedded_x)[sample_mask]
X_embedded_y_sample = np.array(X_embedded_y)[sample_mask]
y_test_sample = y_test_correct[sample_mask]
y_test_class_sample = np.argmax(y_test_sample, axis=1)

# color_list = [class_to_idx_dict[c] for c in y_test_class_sample]
# color_list = [1 for c in y_test_class_sample]
color_list = [int(class_to_idx_dict[c]/20) for c in y_test_class_sample]

In [ ]:
mask_hard_perturbed_lgb = np.logical_or(np.array(queries_lgb) > 10000, np.array(queries_lgb) < 0)
mask_easy_perturbed_lgb = np.logical_and(np.array(queries_lgb) > 0, np.array(queries_lgb) < 50)

In [ ]:
import random

In [ ]:
mask_robust = np.logical_and(sample_mask[:SIZE], mask_hard_perturbed_lgb)

X_embedded_x_sample_robust = np.array(X_embedded_x)[:SIZE][mask_robust]
X_embedded_y_sample_robust = np.array(X_embedded_y)[:SIZE][mask_robust]
y_test_sample_robust = y_test_correct[:SIZE][mask_robust]
y_test_class_sample_robust = np.argmax(y_test_sample_robust, axis=1)

# color_list_robust = [class_to_idx_dict[c] + 10 for c in y_test_class_sample_robust]
# color_list_robust = [int(class_to_idx_dict[c]/50) for c in y_test_class_sample_robust]
color_list_robust = [0 for c in y_test_class_sample_robust]

In [ ]:
mask_not_robust = np.logical_and(sample_mask[:SIZE], mask_easy_perturbed_lgb)

X_embedded_x_sample_not_robust = np.array(X_embedded_x)[:SIZE][mask_not_robust]
X_embedded_y_sample_not_robust = np.array(X_embedded_y)[:SIZE][mask_not_robust]
y_test_sample_not_robust = y_test_correct[:SIZE][mask_not_robust]
y_test_class_sample_not_robust = np.argmax(y_test_sample_not_robust, axis=1)

color_list_not_robust = [class_to_idx_dict[c] + 10 for c in y_test_class_sample_not_robust]
color_list_not_robust = [0 for c in y_test_class_sample_not_robust]

In [ ]:
len(X_embedded_x_sample)

In [ ]:
len(X_embedded_x_sample_not_robust)

In [ ]:
len(X_embedded_x_sample_robust)

In [ ]:
len(queries_lgb)

In [ ]:
plt.figure(figsize=(30,30))
sns.scatterplot(x=X_embedded_x_sample, y=X_embedded_y_sample, hue=color_list)
sns.scatterplot(x=X_embedded_x_sample_robust, y=X_embedded_y_sample_robust, hue=color_list_robust, marker="X", s=100, alpha=0.9)
# sns.scatterplot(x=X_embedded_x_sample_not_robust, y=X_embedded_y_sample_not_robust, hue=color_list_not_robust, marker="X", s=100, alpha=0.9)
plt.show()

In [ ]:
mask_hard_perturbed_lgb = np.logical_or(np.array(queries_lgb) > 1000, np.array(queries_lgb) <0)

In [ ]:
mask_hard_perturbed = np.logical_or(np.array(queries) > 1000, np.array(queries)<0)